In [2]:
#Colab

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# %cd /content/drive/MyDrive/aml_final
# # with open("github_token.txt", "r") as f:
# #  token = f.read()
# #! git clone https://{token}@github.com/Tryner/aml_final.git #clone repo
# %cd aml_final/
# ! git pull
# ! pip install setfit


In [3]:
from collections import Counter
import torch
from datasets import load_dataset, Dataset
from setfit import SetFitModel, Trainer, TrainingArguments
from train.active_learning import ActiveTrainer
from train.active_learning_config import ActiveLearningConfig
from data.dataset_config import DatasetConfig
import pandas as pd
from sklearn.model_selection import train_test_split

c:\Users\UTENTE\Documents\aml_final\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df=pd.read_csv('data\\raw_datasets\\twitter_humor.csv')
# df=pd.read_csv('data/raw_datasets/twitter_humor.csv')

df=df.loc[:,['tweet','humor']]
df=df.rename(columns={'tweet': DatasetConfig.text_column, 'humor': DatasetConfig.label_column})

# Split the DataFrame into features (X) and labels (y)
X = df.drop('label', axis=1)  # Assuming 'label' is the name of your label column
y = df['label']

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataFrames for training and validation
tset = Dataset.from_pandas(pd.concat([X_train, y_train], axis=1))
vset = Dataset.from_pandas(pd.concat([X_valid, y_valid], axis=1))

active_learning_config = ActiveLearningConfig()
dataset_config = DatasetConfig()
train_args = TrainingArguments()

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
def model_init():
    return SetFitModel.from_pretrained("thenlper/gte-small", use_differentiable_head=True, head_params={"out_features": dataset_config.num_classes}).to(device)
def after_train_callback(trainer: Trainer):
    print("#Accuraccy: " + str(trainer.evaluate()))
def dataset_callback(dataset: Dataset):
    print("#Labels: " + str(Counter(dataset[dataset_config.label_column])))

cpu


In [1]:
trainer = ActiveTrainer(
    model_init=model_init, 
    full_train_dataset=tset,
    eval_dataset=vset,
    train_args=train_args,
    active_learning_config=active_learning_config, 
    dataset_config=dataset_config,
    initial_train_subset=None,
    after_train_callback=after_train_callback,
    dataset_callback=dataset_callback,
    )

NameError: name 'ActiveTrainer' is not defined

In [43]:
trainer.train()

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Map: 100%|██████████| 8/8 [00:00<00:00, 667.19 examples/s]
***** Running training *****
  Num examples = 3
  Num epochs = 1
  Total optimization steps = 3
  Total train batch size = 16
  0%|          | 0/3 [00:00<?, ?it/s]


KeyboardInterrupt: 